<a href="https://colab.research.google.com/github/tanuja1708/EEG-emotions/blob/main/seed(s%2Cn%2Ca%2Ca%2Ce)_with_CGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Load Data & Preprocessing

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Load EEG data
data = pd.read_csv("/content/labeled_data(sad,neu,anx,ang,exc).csv")  # Update path if needed

# Extract features (X) and labels (y)
X = data.drop('label', axis=1).values
y = data['label'].values

# Label encoding (convert emotions to integers)
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Normalize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)



# Split into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}")


Train shape: (40728, 13), Test shape: (10182, 13)


#Train Baseline LSTM (Real Data Only)

In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Reshape the input data for LSTM
# Assuming each sample has a single time step and 11 features:
X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])  # Reshape to (40728, 1, 13)
X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])    # Reshape to (10182, 1, 13)
# Define LSTM model
model = Sequential([
    LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2])), # Now input_shape is (1, 11)
    Dense(32, activation='relu'),
    Dense(len(np.unique(y_encoded)), activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train on real data only
history = model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test))

# Evaluate
_, accuracy = model.evaluate(X_test, y_test)
print(f"Baseline Test Accuracy (Real Data Only): {accuracy * 100:.2f}%")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
637/637 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - accuracy: 0.7416 - loss: 0.7638 - val_accuracy: 0.8838 - val_loss: 0.2809
Epoch 2/10
637/637 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.9062 - loss: 0.2428 - val_accuracy: 0.8922 - val_loss: 0.2649
Epoch 3/10
637/637 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.9177 - loss: 0.2087 - val_accuracy: 0.9235 - val_loss: 0.1874
Epoch 4/10
637/637 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9170 - loss: 0.1999 - val_accuracy: 0.9273 - val_loss: 0.1810
Epoch 5/10
637/637 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.9296 - loss: 0.1705 - val_accuracy: 0.9388 - val_loss: 0.1553
Epoch 6/10
637/637 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.9302 - loss: 0.1628 - val_accuracy: 0.9396 - val_loss: 0.1617
Epoch 7/10
637/637 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.9385 - loss: 0.1462 - val_accuracy: 0.9503 - val_loss: 0.1176
Epoch 8/10
637/637 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.9430 - loss: 0.1297 - val_accuracy: 0.

#Conditional GAN (cGAN) - Generator and Discriminator

In [7]:

from tensorflow.keras.layers import Input, Dense, Conv1DTranspose, LeakyReLU, Flatten, Embedding, Concatenate, Reshape
from tensorflow.keras.models import Model, Sequential

latent_dim = 100
n_features = X_train.shape[2]
time_steps = X_train.shape[1]
n_classes = len(np.unique(y_encoded))

# Generator
def build_generator():
    noise_input = Input(shape=(latent_dim,))
    label_input = Input(shape=(1,))
    label_embedding = Flatten()(Embedding(n_classes, latent_dim)(label_input))

    combined = tf.keras.layers.multiply([noise_input, label_embedding])

    x = Dense(128 * time_steps, activation="relu")(combined)
    x = Reshape((time_steps, 128))(x)
    x = Conv1DTranspose(64, kernel_size=3, padding="same", activation="relu")(x)
    x = Conv1DTranspose(n_features, kernel_size=3, padding="same")(x)
    output = Reshape((time_steps, n_features))(x)

    return Model([noise_input, label_input], output)

# Discriminator
def build_discriminator():
    eeg_input = Input(shape=(time_steps, n_features))
    label_input = Input(shape=(1,))
    label_embedding = Flatten()(Embedding(n_classes, time_steps * n_features)(label_input))
    label_embedding = Reshape((time_steps, n_features))(label_embedding)

    combined = Concatenate()([eeg_input, label_embedding])

    x = Flatten()(combined)
    x = Dense(128, activation=LeakyReLU(0.2))(x)
    x = Dense(1, activation='sigmoid')(x)

    return Model([eeg_input, label_input], x)


#Compile & Train GAN

In [8]:
generator = build_generator()
discriminator = build_discriminator()

discriminator.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

z = Input(shape=(latent_dim,))
label = Input(shape=(1,))
fake_eeg = generator([z, label])
discriminator.trainable = False
validity = discriminator([fake_eeg, label])

combined = Model([z, label], validity)
combined.compile(loss='binary_crossentropy', optimizer='adam')


#GAN Training Loop


In [9]:
import random
import numpy as np # Importing numpy

def train_gan(epochs=5000, batch_size=64):
    half_batch = batch_size // 2

    for epoch in range(epochs):
        # Train Discriminator
        idx = np.random.randint(0, X_train.shape[0], half_batch)
        real_eegs = X_train[idx]
        real_labels = y_train[idx]

        # The real_eegs already have the correct shape (half_batch, 1, n_features)
        # because X_train was reshaped earlier. Remove the reshape line:
        # real_eegs = real_eegs.reshape(real_eegs.shape[0], 1, real_eegs.shape[1])

        noise = np.random.normal(0, 1, (half_batch, latent_dim))
        fake_eegs = generator.predict([noise, real_labels])

        d_loss_real = discriminator.train_on_batch([real_eegs, real_labels], np.ones((half_batch, 1)))
        d_loss_fake = discriminator.train_on_batch([fake_eegs, real_labels], np.zeros((half_batch, 1)))
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Train Generator
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        sampled_labels = np.random.randint(0, n_classes, batch_size)
        valid_y = np.ones((batch_size, 1))

        g_loss = combined.train_on_batch([noise, sampled_labels], valid_y)

        if epoch % 1000 == 0:
            print(f"{epoch} [D loss: {d_loss[0]}, acc.: {100*d_loss[1]}%] [G loss: {g_loss}]")

train_gan()

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


/usr/local/lib/python3.11/dist-packages/keras/src/backend/tensorflow/trainer.py:82: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


Streaming output truncated to the last 5000 lines.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step

#Generate Synthetic Data for Augmented Training

In [15]:
def generate_synthetic_data(n_samples=10000):
    noise = np.random.normal(0, 1, (n_samples, latent_dim))
    labels = np.random.randint(0, n_classes, n_samples)
    synthetic_eeg = generator.predict([noise, labels])
    return synthetic_eeg, labels

synthetic_X, synthetic_y = generate_synthetic_data(10000)

# Combine Real + Synthetic Data
X_augmented = np.vstack([X_train, synthetic_X])
y_augmented = np.concatenate([y_train, synthetic_y])

print(f"Augmented Data Shape: {X_augmented.shape}, Labels Shape: {y_augmented.shape}")


313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Augmented Data Shape: (50728, 1, 13), Labels Shape: (50728,)


#Train LSTM on Combined Data (Real + Synthetic)

In [17]:
model_augmented = Sequential([
    LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dense(32, activation='relu'),
    Dense(n_classes, activation='softmax')
])

model_augmented.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train with Augmented Data
history_aug = model_augmented.fit(X_augmented, y_augmented, epochs=50, batch_size=64, validation_data=(X_test, y_test))

# Evaluate
_, accuracy_aug = model_augmented.evaluate(X_test, y_test)
print(f"Augmented Test Accuracy (Real + Synthetic): {accuracy_aug * 100:.2f}%")


Epoch 1/50
793/793 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.6462 - loss: 0.8958 - val_accuracy: 0.8907 - val_loss: 0.2761
Epoch 2/50
793/793 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.7718 - loss: 0.5050 - val_accuracy: 0.9104 - val_loss: 0.2198
Epoch 3/50
793/793 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.7795 - loss: 0.4792 - val_accuracy: 0.9248 - val_loss: 0.1884
Epoch 4/50
793/793 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.7835 - loss: 0.4669 - val_accuracy: 0.9323 - val_loss: 0.1663
Epoch 5/50
793/793 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.7889 - loss: 0.4517 - val_accuracy: 0.9308 - val_loss: 0.1662
Epoch 6/50
793/793 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.7881 - loss: 0.4471 - val_accuracy: 0.9394 - val_loss: 0.1509
Epoch 7/50
793/793 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.7958 - loss: 0.4336 - val_accuracy: 0.9499 - val_loss: 0.1277
Epoch 8/50
793/793 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.7975 - loss: 0.4214 - val_accuracy: 0.

In [18]:
generator.save('/content/generator_model.h5')
print("✅ Generator saved successfully!")


✅ Generator saved successfully!


In [19]:
discriminator.save('/content/discriminator_model.h5')
print("✅ Discriminator saved successfully!")


✅ Discriminator saved successfully!


In [21]:
model_augmented.save('/content/lstm_model.h5')
print("✅ LSTM classifier saved successfully!")

✅ LSTM classifier saved successfully!


In [29]:
import joblib

# Assuming `scaler` was fit on training data (X_train)
joblib.dump(scaler, 'scaler.pkl')
print("Scaler saved as scaler.pkl")


Scaler saved as scaler.pkl


In [30]:
joblib.dump(label_encoder, 'label_encoder.pkl')
print("Label encoder saved as label_encoder.pkl")


Label encoder saved as label_encoder.pkl


In [22]:
from tensorflow.keras.models import load_model

lstm_model = load_model('/content/lstm_model.h5')
print("✅ LSTM model loaded successfully!")


✅ LSTM model loaded successfully!


In [23]:
from tensorflow.keras.models import load_model

lstm_model = load_model('/content/lstm_model.h5')
print("✅ LSTM model loaded successfully!")


✅ LSTM model loaded successfully!


In [69]:
import numpy as np
import pandas as pd
import joblib  # For loading scaler and label encoder
from tensorflow.keras.models import load_model

# 1. Load Saved Model, Scaler, and Label Encoder
lstm_model = load_model("/content/lstm_model.h5")
scaler = joblib.load("/content/scaler.pkl")
label_encoder = joblib.load("/content/label_encoder.pkl")

# 2. Generate New Random EEG Data (5 time steps, 13 EEG channels) - fresh every run
new_data = np.random.uniform(-5, 5, (5, 13))  # EEG values between -5 and 5

# 3. Print New Random Input Data (as DataFrame for easier viewing)
columns = [f"Channel{i+1}" for i in range(13)]  # 13 EEG channels
new_data_df = pd.DataFrame(new_data, columns=columns)
print("Randomly Generated EEG Input Data (new each run):")
print(new_data_df)

# 4. Normalize New Data Using Saved Scaler
new_data_scaled = scaler.transform(new_data)

# Debug: Check scaled data
print("\nScaled Input Data (after normalization):")
print(pd.DataFrame(new_data_scaled, columns=columns))

# 5. Reshape to (1 sample, 5 time steps, 13 features) for LSTM input
new_data_scaled = new_data_scaled.reshape(1, 5, 13)

# 6. Predict Emotion
predicted_prob = lstm_model.predict(new_data_scaled)

# Debug: Print predicted probabilities for each class
emotion_classes = label_encoder.classes_
print("\nPredicted Probabilities for Each Emotion:")
for emotion, prob in zip(emotion_classes, predicted_prob[0]):
    print(f"{emotion}: {prob:.4f}")

# 7. Find Predicted Class
predicted_class = np.argmax(predicted_prob, axis=1)

# 8. Decode to Emotion Label
predicted_emotion = label_encoder.inverse_transform(predicted_class)[0]

# 9. Output Predicted Emotion
print(f"\nPredicted Emotion: {predicted_emotion}")


Randomly Generated EEG Input Data (new each run):
   Channel1  Channel2  Channel3  Channel4  Channel5  Channel6  Channel7  \
0 -4.034655 -2.581062 -4.875960 -0.312320 -1.987347  0.983575 -2.027622   
1  0.932216  3.923025 -3.146698 -4.210308 -2.604898  2.945783 -4.653297   
2  0.989785 -3.850670 -4.061427  4.096268  1.692003  3.292868  3.789789   
3  1.019234  3.925233 -0.566200  1.070896  1.313076  0.916972  2.026338   
4  4.505311  1.006511  2.435939  0.062661  1.341040 -4.290678 -2.456084   

   Channel8  Channel9  Channel10  Channel11  Channel12  Channel13  
0 -2.000809  2.431932  -4.518572   4.028950   3.522639   1.678047  
1  0.828068  4.954375   3.556961   0.214457  -4.363591   3.313735  
2  0.717724  0.174464  -0.695726  -1.830534  -0.654040   2.738797  
3 -2.625665  0.123638  -3.957752  -1.154887  -0.123329   1.522243  
4 -1.381469 -0.275066  -4.543513  -3.599759  -2.231857   4.715327  

Scaled Input Data (after normalization):
    Channel1   Channel2  Channel3   Channel4  Cha

In [4]:
import time
import numpy as np
import pandas as pd
import joblib  # For loading scaler and label encoder
from tensorflow.keras.models import load_model
from google.colab import files  # Import files from Google Colab


# Function to simulate getting the latest EEG window (replace with your actual data source)
def get_latest_eeg_window():
    """Simulates getting a 2-second window of EEG data.

    Replace this with your logic to read from your EEG device/data source.

    Returns:
        numpy.ndarray: A 2D array of shape (5, 13) representing 5 time steps and 13 channels.
    """
    # For demonstration, generate random data
    return np.random.uniform(-5, 5, (5, 13))

# Function to predict emotion using the loaded model (replace with your model)
def predict_emotion(eeg_window):
    """Predicts emotion based on the provided EEG window.

    Args:
        eeg_window (numpy.ndarray): A 2D array of shape (5, 13) representing EEG data.

    Returns:
        str: The predicted emotion label.
    """
    # 1. Load Saved Model, Scaler, and Label Encoder (if not already loaded)
    # Before loading, upload the necessary files if they are not present
    try:
        lstm_model = load_model("/content/lstm_model(2).h5")
    except FileNotFoundError:
        print("lstm_model.h5 not found. Please upload it.")
        files.upload()  # Upload lstm_model.h5
        lstm_model = load_model("/content/lstm_model (2).h5")

    try:
        scaler = joblib.load("/content/scaler.pkl")
    except FileNotFoundError:
        print("scaler.pkl not found. Please upload it.")
        files.upload()  # Upload scaler.pkl
        scaler = joblib.load("/content/scaler.pkl")

    try:
        label_encoder = joblib.load("/content/label_encoder.pkl")
    except FileNotFoundError:
        print("label_encoder.pkl not found. Please upload it.")
        files.upload()  # Upload label_encoder.pkl
        label_encoder = joblib.load("/content/label_encoder.pkl")

    # 2. Normalize EEG data using the loaded scaler
    eeg_window_scaled = scaler.transform(eeg_window)

    # 3. Reshape to (1 sample, 5 time steps, 13 features) for LSTM input
    eeg_window_scaled = eeg_window_scaled.reshape(1, 5, 13)

    # 4. Predict using the LSTM model
    predicted_prob = lstm_model.predict(eeg_window_scaled)

    # 5. Find Predicted Class and decode to emotion label
    predicted_class = np.argmax(predicted_prob, axis=1)
    predicted_emotion = label_encoder.inverse_transform(predicted_class)[0]

    return predicted_emotion


# Function to simulate sending emotion to the bot (replace with your logic)
def send_emotion_to_bot(emotion):
    """Simulates sending the emotion to a bot.

    Replace with your logic to send data via WebSocket or HTTP request.

    Args:
        emotion (str): The emotion label to send.
    """
    print(f"Sending emotion to bot: {emotion}")


while True:
    eeg_window = get_latest_eeg_window()  # latest 2-second EEG data
    emotion = predict_emotion(eeg_window)  # your trained model

    send_emotion_to_bot(emotion)  # via WebSocket or HTTP request

    if emotion == "neutral":
        print("Emotion is neutral, ending conversation.")
        break

    time.sleep(2)  # wait for next window

lstm_model.h5 not found. Please upload it.


Saving lstm_model (2).h5 to lstm_model (2) (2).h5
scaler.pkl not found. Please upload it.


Saving scaler.pkl to scaler.pkl
label_encoder.pkl not found. Please upload it.


Saving label_encoder.pkl to label_encoder.pkl
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 232ms/step
Sending emotion to bot: Neutral
lstm_model.h5 not found. Please upload it.


KeyboardInterrupt: 

In [68]:
import numpy as np
import pandas as pd

# Define emotions
emotions = ["sad", "neutral", "anxious", "angry", "excited"]

# Function to generate synthetic EEG-like data for each emotion
def generate_emotion_eeg(emotion):
    eeg = np.zeros((5, 13))  # 5 time steps, 13 channels

    if emotion == "sad":
        # Higher delta power (1.5 - 2.5 simulated as lower frequency band power)
        eeg += np.random.uniform(1.5, 2.5, (5, 13))

    elif emotion == "neutral":
        # Balanced power (baseline noise)
        eeg += np.random.uniform(-0.5, 0.5, (5, 13))

    elif emotion == "anxious":
        # Higher theta power (4 - 8 Hz, but here just elevated random noise for simplicity)
        eeg += np.random.uniform(2, 3, (5, 13))

    elif emotion == "angry":
        # Higher gamma power (above 30 Hz - represented by stronger random spikes)
        eeg += np.random.uniform(3, 4, (5, 13))

    elif emotion == "excited":
        # Higher beta power (13 - 30 Hz, high mid-range power)
        eeg += np.random.uniform(2.5, 3.5, (5, 13))

    # Add small random noise to all channels to simulate real EEG noise
    eeg += np.random.normal(0, 0.2, (5, 13))

    return eeg

# Generate and print example data for each emotion
for emotion in emotions:
    eeg_data = generate_emotion_eeg(emotion)
    print(f"\nEEG Input for Emotion: {emotion}")
    eeg_df = pd.DataFrame(eeg_data, columns=[f"Channel{i+1}" for i in range(13)])
    print(eeg_df.round(2))



EEG Input for Emotion: sad
   Channel1  Channel2  Channel3  Channel4  Channel5  Channel6  Channel7  \
0      1.92      1.98      1.82      2.06      2.30      2.27      1.84   
1      2.19      2.23      2.29      2.23      2.43      1.71      2.14   
2      2.77      1.74      1.72      2.31      1.68      1.74      1.69   
3      1.60      1.69      1.85      1.78      1.76      2.27      1.97   
4      2.16      1.36      2.17      2.17      2.08      2.14      2.24   

   Channel8  Channel9  Channel10  Channel11  Channel12  Channel13  
0      1.58      1.67       2.31       1.66       2.14       2.16  
1      1.40      1.87       2.38       2.26       2.27       1.89  
2      2.10      1.91       1.57       2.12       2.06       1.82  
3      1.94      2.15       1.62       2.45       2.10       2.32  
4      1.98      1.93       2.23       1.69       2.28       2.64  

EEG Input for Emotion: neutral
   Channel1  Channel2  Channel3  Channel4  Channel5  Channel6  Channel7  \
0     

In [5]:
import random
import time

# Sample emotion detection function (simulates EEG input features to emotion mapping)
def predict_emotion():
    emotions = ["Happy", "Sad", "Stressed", "Neutral", "Anxious"]
    return random.choice(emotions)

# Predefined chatbot responses
chatbot_responses = {
    "Happy": "Awesome! Love to see you happy! Anything fun planned?",
    "Sad": "I'm here for you. Do you want me to share something uplifting?",
    "Stressed": "Hey, I noticed you might be feeling a bit overwhelmed. Want to talk about what's on your mind?",
    "Neutral": "I see you're feeling calm now. Let me know if you need anything.",
    "Anxious": "It's okay to feel anxious sometimes. Let's take a deep breath together."
}

# Parameters
neutral_count = 0  # To track consecutive neutral detections
consecutive_neutral_threshold = 3  # End chat after 3 consecutive neutrals

# Simulate continuous EEG processing & chatbot conversation
print("🔵 Chatbot started... Monitoring emotions.\n")

while True:
    emotion = predict_emotion()

    # Show detected emotion
    print(f"🧠 Detected Emotion: {emotion}")

    # Chatbot response
    response = chatbot_responses.get(emotion, "I'm not sure how you're feeling. Let's talk.")
    print(f"🤖 Chatbot: {response}\n")

    # Check for consecutive neutral emotion
    if emotion == "Neutral":
        neutral_count += 1
    else:
        neutral_count = 0  # Reset if any non-neutral emotion detected

    # End session if neutral detected 3 times in a row
    if neutral_count >= consecutive_neutral_threshold:
        print("✅ Emotion stabilized to 'Neutral' for 3 consecutive windows. Ending chat.\n")
        break

    # Simulate 2-second EEG window (adjust as per actual data frequency)
    time.sleep(2)

print("🔵 Chatbot session ended.")


🔵 Chatbot started... Monitoring emotions.

🧠 Detected Emotion: Sad
🤖 Chatbot: I'm here for you. Do you want me to share something uplifting?

🧠 Detected Emotion: Neutral
🤖 Chatbot: I see you're feeling calm now. Let me know if you need anything.

🧠 Detected Emotion: Neutral
🤖 Chatbot: I see you're feeling calm now. Let me know if you need anything.

🧠 Detected Emotion: Sad
🤖 Chatbot: I'm here for you. Do you want me to share something uplifting?

🧠 Detected Emotion: Anxious
🤖 Chatbot: It's okay to feel anxious sometimes. Let's take a deep breath together.

🧠 Detected Emotion: Stressed
🤖 Chatbot: Hey, I noticed you might be feeling a bit overwhelmed. Want to talk about what's on your mind?

🧠 Detected Emotion: Anxious
🤖 Chatbot: It's okay to feel anxious sometimes. Let's take a deep breath together.

🧠 Detected Emotion: Stressed
🤖 Chatbot: Hey, I noticed you might be feeling a bit overwhelmed. Want to talk about what's on your mind?

🧠 Detected Emotion: Happy
🤖 Chatbot: Awesome! Love to

KeyboardInterrupt: 